# Question 1

**Task 1.1**
Return to the writeup you created for project #2. Given what you've just learned about Natural Language Processing, tweak / refine your writeup to incorporate those techniques into your" fake news detection" program.<br>
Now since we have CountVectorizer and TfidfVectorizer we donot need to construct a vocubulary to compute the features. Now once we have obtained our text data we need to clean the data using word tokenizer removing stop words and punctuations using nltk stop words and punctions from the strings. Once we obtained this data we can use the Stemming / Lematization technique to make  different derivative form of words to a single word.<br>
Finall, once we have obtained the cleaned , stemmed data set we can use either Countvectorizer or TfidfVectorizer based on the performance of our classification model's output.If it is giving better for CountVectorizer then we use it otherwise we use TfidfVectorizer.<br>
Finally , we can use different model based on performance like Logistic Regression, Random Forest , Gradient Boosting Classifier etc for our prediction.<br>
So we take a combination of feature extraction models( i.e. CountVectorizer,TfidfVectorizer) and prediction models (i.e. Logistic Regression, Random Forest , Gradient Boosting Classifier) and check for best performance and choose that combination with best performance over the test data.

# Question 2

In [1]:
import mysql.connector
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="password",
  database="news_reports"
)

Creating Table to dump values from news API

In [2]:
mycursor = mydb.cursor()
mycursor.execute("CREATE TABLE articles ( title text(255))")

**Task 2.1**
A standalone Python application (that you create), that streams in news headlines/articles from one or more of the APIs you wrote about, and dumps the results into a relational database of your choosing.

In [332]:
import requests
import json

query="INSERT INTO articles (title) VALUES (%s)"
def get_stream(url):
#     global decoded_line
    r = requests.get(url, stream=True)
    for line in r.iter_lines(chunk_size=10):
        if line:
            decoded_line = line.decode('utf-8')
            dat=json.loads(decoded_line)["articles"]
        for lines in dat:
            mycursor = mydb.cursor()
            val=lines["description"]
            if (val,) not in records:
                mycursor.execute(query,(val,))
                mydb.commit()
url='https://newsapi.org/v2/everything?q=google&apiKey=6ad7a293f5534b97951d0377356ae7dd'
get_stream(url)

In [333]:
mycursor = mydb.cursor(buffered=True)
mycursor.execute("SELECT * FROM articles")
records = mycursor.fetchall()
type(records)

list

In [334]:
len(records)

61

**Task 2.2.**
A set of commands you can use to dump the database contents into a CSV, containing only the columns that matter to you.

In [336]:
import pandas as pd
my_dict = {'articles': records}
dump_db_to_df=pd.DataFrame(my_dict)
dump_db_to_df.to_csv('dump_db.csv',index=False)

**Task 2.3.**
The notebook formulas that can analyze the spam news dataset(s) and learn from them.

In [52]:
import nltk
from nltk.tokenize import word_tokenize 
from nltk.stem import SnowballStemmer
import string

In [242]:
df=pd.read_csv('news_data.csv',usecols=['title','label'])

In [243]:
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords_set=set(stopwords)
punctuation_set=set(string.punctuation)

In [300]:
def data_processor(doc):
    stops=stopwords
    stemmer=SnowballStemmer('english')
    doc = word_tokenize(doc.lower())
    tokens = [tok for tok in doc if tok not in punctuation_set]
    tokens = [tok for tok in tokens if tok]
    if stops:
        tokens = [tok for tok in tokens if (tok not in stops)]
    if stemmer:
        tokens = [stemmer.stem(tok) for tok in tokens]
    sentence=' '.join([word for word in tokens])
    return sentence

In [301]:
df['clr_title']=df.title.apply(data_processor)

In [302]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
count_vect= TfidfVectorizer()
X= count_vect.fit_transform(df['clr_title'])
y=df.label
X_train, X_test, y_train, y_test= train_test_split(X,y,test_size=0.1)

In [303]:
from sklearn.linear_model import LogisticRegression
lg= LogisticRegression(C=5)

lg.fit(X_train,y_train)
y_pred=lg.predict(X_test)
lg.score(X_test,y_test)

0.955456570155902

In [304]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[2019,   85],
       [ 115, 2271]], dtype=int64)

**Task 2.4.**
The notebook formulas that can import the CSV contents from step 2, and analyze the new headlines/articles, then give them a truthfulness rating. For example: "Likely true", "likely false", etc

In [338]:
db_df=pd.read_csv('dump_db.csv')
db_df['articles']=db_df.articles.apply(lambda x:x[2:-1])
db_df['clr_articles']=db_df['articles'].apply(data_processor)
X_db=count_vect.transform(db_df['clr_articles'])
db_predict=lg.predict(X_db)
db_df['label']=db_predict
db_df['Fake/True']=db_df['label'].apply(lambda x:'Likely False' if x==1 else 'Likely True')
db_df[['articles','Fake/True']]

,articles,Fake/True
40,Scoreboard at close of play of 3rd t20i betwee...,Likely True
41,YouTube livestreaming has been a big promise o...,Likely True
42,Google announced this morning it’s adding supp...,Likely True
43,Several new features are showing up in the Goo...,Likely False
44,"Finally, Google has updated its useful Authent...",Likely False
45,"Last month, Google announced End-to-End encryp...",Likely False
46,Google search’s new online shopping feature ma...,Likely True
47,Google is giving Pixel 3 devices and later mod...,Likely False
48,Google Podcasts just became more alluring if y...,Likely False
49,Google has expanded the number of augmented re...,Likely False
